<a href="https://colab.research.google.com/github/dk-wei/python-multiprocessing/blob/main/Python_Multiprocessing_Ray.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
#!pip install ray[default]

In [10]:
import multiprocessing as mp
from tqdm.notebook import tqdm
import ray
import time

# Parallelism with Tasks

## Independent Task

To turn a Python function `f` into a “remote function” (a function that can be executed remotely and asynchronously), we declare the function with the `@ray.remote` decorator. Then function invocations via `f.remote()` will immediately return futures (a future is a reference to the eventual output), and the actual function execution will take place in the background (we refer to this execution as a **task**).

先将list进行chuck，再进行parallel computing

In [40]:
org_list = [3,4,5,6,5,4,3,2,1,3,4,4,3,2,4,6,2,1,32,4,55,3,2,1,4,6,78,8,5,6,0,9,8,6,3,5,6,7,7,8,8,4,2]

def split(a, n):
  '''
  a: 需要chunk的large list
  n: 希望chunk出多少个sublist
  '''
  k, m = divmod(len(a), n)
  split_data = [a[i*k+min(i, m):(i+1)*k+min(i+1, m)] for i in range(n)]

  # 没办法，我们得在每个sublist前面做好index的标记，最后拼接result的时候会用到
  split_data_order_number = [[i, v] for i, v in enumerate(split_data)]

  return split_data_order_number
        
sub_list = split(org_list,6)

sub_list

[[0, [3, 4, 5, 6, 5, 4, 3, 2]],
 [1, [1, 3, 4, 4, 3, 2, 4]],
 [2, [6, 2, 1, 32, 4, 55, 3]],
 [3, [2, 1, 4, 6, 78, 8, 5]],
 [4, [6, 0, 9, 8, 6, 3, 5]],
 [5, [6, 7, 7, 8, 8, 4, 2]]]

In [41]:
%%time

# Start Ray.
ray.init(num_cpus=8)  # 设置进程数

@ray.remote
def f(x):
  #time.sleep(1)
  index = x[0]
  value = x[1]
  init_value = 0
  for item in value:
    init_value += item

  return index, init_value

# Start 4 tasks in parallel.
result_ids = []
for i in sub_list:
    result_ids.append(f.remote(i))

2021-05-07 02:32:25,801	INFO services.py:1269 -- View the Ray dashboard at http://127.0.0.1:8265


CPU times: user 59.7 ms, sys: 66.3 ms, total: 126 ms
Wall time: 4.43 s


In [42]:
# Wait for the tasks to complete and retrieve the results.
# With at least 4 cores, this will take 1 second.
# 只有到get这步才会开始运算
results = ray.get(result_ids)  # [0, 1, 2, 3]

results

[(0, 32), (1, 21), (2, 103), (3, 104), (4, 37), (5, 42)]

In [39]:
ray.shutdown()

从代码上看，远比python原生`Multiprocessing`模块要简单，而且速度会更快。

## Task Dependencies

Tasks can also depend on other tasks. Below, the `multiply_matrices` task uses the outputs of the two `create_matrix` tasks, so it will not begin executing until after the first two tasks have executed. The outputs of the first two tasks will automatically be passed as arguments into the third task and the futures will be replaced with their corresponding values). In this manner, tasks can be composed together with arbitrary DAG dependencies.



几个function之间存在依赖关系

In [7]:
import numpy as np

@ray.remote
def create_matrix(size):
    return np.random.normal(size=size)

@ray.remote
def multiply_matrices(x, y):
    return np.dot(x, y)

x_id = create_matrix.remote([1000, 1000])
y_id = create_matrix.remote([1000, 1000])
z_id = multiply_matrices.remote(x_id, y_id)

# Get the results.
z = ray.get(z_id)

## Aggregating Values Efficiently

Task dependencies can be used in much more sophisticated ways. For example, suppose we wish to aggregate 8 values together. This example uses integer addition, but in many applications, aggregating large vectors across multiple machines can be a bottleneck. In this case, changing a single line of code can change the aggregation’s running time from linear to logarithmic in the number of values being aggregated.

![](https://miro.medium.com/max/1750/1*vHz3troEmr4uLns0V8VmdA.jpeg)

As described above, to feed the output of one task as an input into a subsequent task, simply pass the future returned by the first task as an argument into the second task. This task dependency will automatically be taken into account by Ray’s scheduler. The second task will not execute until the first task has finished, and the output of the first task will automatically be shipped to the machine on which the second task is executing.

In [43]:
import time

@ray.remote
def add(x, y):
    time.sleep(1)
    return x + y

# Aggregate the values slowly. This approach takes O(n) where n is the
# number of values being aggregated. In this case, 7 seconds.
id1 = add.remote(1, 2)
id2 = add.remote(id1, 3)
id3 = add.remote(id2, 4)
id4 = add.remote(id3, 5)
id5 = add.remote(id4, 6)
id6 = add.remote(id5, 7)
id7 = add.remote(id6, 8)
result = ray.get(id7)

# Aggregate the values in a tree-structured pattern. This approach
# takes O(log(n)). In this case, 3 seconds.
id1 = add.remote(1, 2)
id2 = add.remote(3, 4)
id3 = add.remote(5, 6)
id4 = add.remote(7, 8)
id5 = add.remote(id1, id2)
id6 = add.remote(id3, id4)
id7 = add.remote(id5, id6)
result = ray.get(id7)

The above code is very explicit, but note that both approaches can be implemented in a more concise fashion using while loops.

In [44]:
# Slow approach.
values = [1, 2, 3, 4, 5, 6, 7, 8]
while len(values) > 1:
    values = [add.remote(values[0], values[1])] + values[2:]
result = ray.get(values[0])


# Fast approach.
values = [1, 2, 3, 4, 5, 6, 7, 8]
while len(values) > 1:
    values = values[2:] + [add.remote(values[0], values[1])]
result = ray.get(values[0])

# From Classes to Actors

如果想写程序，全部放入`Class`，可以用`Actors`

In [45]:
@ray.remote
class Counter(object):
    def __init__(self):
        self.x = 0
    
    def inc(self):
        self.x += 1
    
    def get_value(self):
        return self.x

# Create an actor process.
c = Counter.remote()

# Check the actor's counter value.
print(ray.get(c.get_value.remote()))  # 0

# Increment the counter twice and check the value again.
c.inc.remote()
c.inc.remote()
print(ray.get(c.get_value.remote()))  # 2

0
2


## Actor Handles

In [47]:
import time


@ray.remote
class MessageActor(object):
    def __init__(self):
        self.messages = []
    
    def add_message(self, message):
        self.messages.append(message)
    
    def get_and_clear_messages(self):
        messages = self.messages
        self.messages = []
        return messages


# Define a remote function which loops around and pushes
# messages to the actor.
@ray.remote
def worker(message_actor, j):
    for i in range(100):
        time.sleep(1)
        message_actor.add_message.remote(
            "Message {} from worker {}.".format(i, j))


# Create a message actor.
message_actor = MessageActor.remote()

# Start 3 tasks that push messages to the actor.
[worker.remote(message_actor, j) for j in range(3)]

# Periodically get the messages and print them.
for _ in range(10):
    new_messages = ray.get(message_actor.get_and_clear_messages.remote())
    print("New messages:", new_messages)
    time.sleep(1)

# This script prints something like the following:
# New messages: []
# New messages: ['Message 0 from worker 1.', 'Message 0 from worker 0.']
# New messages: ['Message 0 from worker 2.', 'Message 1 from worker 1.', 'Message 1 from worker 0.', 'Message 1 from worker 2.']
# New messages: ['Message 2 from worker 1.', 'Message 2 from worker 0.', 'Message 2 from worker 2.']
# New messages: ['Message 3 from worker 2.', 'Message 3 from worker 1.', 'Message 3 from worker 0.']
# New messages: ['Message 4 from worker 2.', 'Message 4 from worker 0.', 'Message 4 from worker 1.']
# New messages: ['Message 5 from worker 2.', 'Message 5 from worker 0.', 'Message 5 from worker 1.']

2021-05-07 02:48:31,251	WARNING worker.py:1115 -- WARNING: 6 PYTHON workers have been started on a node of the id: 910eb7418957359ac8f4292c4235dc0ebea06ee9c15bf753528d6d97 and address: 172.28.0.2. This could be a result of using a large number of actors, or it could be a consequence of using nested tasks (see https://github.com/ray-project/ray/issues/3644) for some a discussion of workarounds.
2021-05-07 02:48:31,987	WARNING worker.py:1115 -- WARNING: 9 PYTHON workers have been started on a node of the id: 910eb7418957359ac8f4292c4235dc0ebea06ee9c15bf753528d6d97 and address: 172.28.0.2. This could be a result of using a large number of actors, or it could be a consequence of using nested tasks (see https://github.com/ray-project/ray/issues/3644) for some a discussion of workarounds.
2021-05-07 02:48:32,727	WARNING worker.py:1115 -- WARNING: 11 PYTHON workers have been started on a node of the id: 910eb7418957359ac8f4292c4235dc0ebea06ee9c15bf753528d6d97 and address: 172.28.0.2. This cou

New messages: []
New messages: ['Message 0 from worker 0.', 'Message 0 from worker 1.', 'Message 0 from worker 2.']
New messages: ['Message 1 from worker 0.', 'Message 1 from worker 1.', 'Message 1 from worker 2.']
New messages: ['Message 2 from worker 0.', 'Message 2 from worker 1.', 'Message 2 from worker 2.']
New messages: ['Message 3 from worker 0.', 'Message 3 from worker 1.', 'Message 3 from worker 2.']
New messages: ['Message 4 from worker 0.', 'Message 4 from worker 1.', 'Message 4 from worker 2.']
New messages: ['Message 5 from worker 0.', 'Message 5 from worker 1.', 'Message 5 from worker 2.']
New messages: ['Message 6 from worker 0.', 'Message 6 from worker 1.', 'Message 6 from worker 2.']
New messages: ['Message 7 from worker 0.', 'Message 7 from worker 1.', 'Message 7 from worker 2.']
New messages: ['Message 8 from worker 0.', 'Message 8 from worker 1.', 'Message 8 from worker 2.']


Actors are extremely powerful. They allow you to take a Python class and instantiate it as a microservice which can be queried from other actors and tasks and even other applications.